# Questao 1

Escolha uma das metodologias que voce implementou na segunda lista para gerar correspondˆencias entre um par de imagens. Aplique-a em 5 pares de imagens (com sobreposi ̧c ̃ao) para calcular suas homografias, e aplique-as para gerar panoramas entre os pares de imagens (um panorama por par). Obs.: nessa questao, nao  ́e permitido usar a API de alto nıvel Stitcher. Dica: use a funcao warpPerspective da OpenCV.

# Questao 2

Repita a questao anterior com 5 trios de imagens (com sobreposicao 2 a 2), alinhando as imagens no plano da primeira imagem. Repita o mesmo alinhando no plano da segunda imagem, e da terceira imagem. Note que aqui sera necessario compor as transformacoes de homografia em alguns casos, ou calcular inversas.

# Questao 3

Use homografias para trocar as bandeiras dos paıses na imagem abaixo (e em anexo no classroom) por outras bandeiras. Preencha tambem a placa de publicidade onde ha o nome ”London 2012” por outra imagem de publicidade de sua escolha.

Dica: leia este tutorial, https://medium.com/acmvit/how-to-project-an-image-in-perspective-view-of-a-background-image-opencv-python-d101bdf966bc

# Questao 4

Leia o seguinte tutorial de calibracao de camera: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html Voce vai precisar de um tabuleiro de xadrez (pode imprimir numa folha A4, e colar num papelao ou emplastificar para a geometria ficar fixa). Meca as dimensoes do seu tabuleiro para calibrar a camera, considerando que o tabuleiro sempre esta no plano z = 0, e que o canto inferior esquerdo do tabuleiro e a origem (0, 0, 0). Apos calibrar a camera, vamos incluir um objeto virtual na imagem. Considere a seguinte equacao parametrica do cırculo centrado no ponto (1.5W, 1.5H, 0), com raio r = 0.5W e contido no plano z = 0, onde H e W sao a as medidas da altura e largura do tabuleiro:

p(θ) = (r cos θ + 1.5W, r sen θ + 1.5H, 0).

Se assegure de que o cırculo apareca na imagem, de acordo com a posicao do xadrez na imagem. Para desenhar o cırculo, varie o valor do angulo θ entre 0 e 2π para amostrar alguns pontos, e projete-os na imagem. Repita isso 3 vezes, variando o angulo entre o vetor normal do tabuleiro e o eixo principal da camera.